<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C11-Offensive_Forensics/Custom_Volatility_Plug-Ins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Custom Volatility Plug-Ins__*

We've just seen how we can use the Volatility plug-ins to analyze a VM snapshot for existing vulnerabilities, profile the user by checking the commands and processes in use, and dump the password hashes. But since you can write your own custom plug-ins, only your imagination limits what you can do with Volatility. If you need additional information based on clues found from the standard plug-ins, you can make a plug-in of your own.

The Volatility team has made it easy to create a plug-in, as long as you follow their pattern. You can even have your new plug-in call other plug-ins to make your job even easier.

Let's take a look at the skeleton of a typical plug-in:

In [ ]:
#imports ...

class CmdLine(interfaces.plugin.PluginInterface): #[1]
    @classmethod
    def get_requirements(cls): #[2]
        pass

    def run(self): #[3]
        pass

    def generator(self, procs): #[4]
        pass

The main steps here are to create your new class to inherit from the __PluginInterface__ __[1]__, define your plug-in's requirements __[2]__, define the __run__ method __[3]__, and define the __generator__ method __[4]__. The __generator__ method is optional, but separating it from the __run__ method is a useful pattern you'll see in many plug-ins. By separating it and using it as a Python generator, you can get faster results and make your code easier to understand.

Let's follow this general pattern to create a custom plug-in that will check for processes that are not protected by Address Space Layout Randomization (__ASLR__). ASLR mixes up the address space of a vulnerable process, which affects the virtual memory location of heaps, stacks, and other operating system allocations. That means that exploit writers cannot determine how the address space of the victim process is laid out at the time of attack. Windows Vista was the first Windows release with ASLR support. In older memory images like Windows XP, you won't see ASLR protection enabled by default. Now, with recent machines (Windows 10), almost all processes are protected.

ASLR doesn't mean the attacker is out of business, but it makes the job much more complicated. As a first step in reconnoitering the processes, we'll create a plug-in to check if a process is protected by ASLR.

Let's get started. Create a directory called __plugins__. Under that directory, create a __windows__ directory to contain your custom plug-ins for Windows machines. If you create plug-ins to target a Mac or Linux machine, create a directory named __mac__ or __linux__, respectively.

Now, in the __plugins/windows__ directory, let's write our ASLR-checking plug-in, __aslrcheck.py__:

In [ ]:
# Search all processes and check for ASLR protection

from typing import Callable, List

from volatility.framework import constants, exceptions, interfaces, renderers
from volatility.framework.configuration import requirements
from volatility.framework.renderers import format_hints
from volatility.framework.symbols import intermed
from volatility.framework.symbols.windows import extensions
from volatility.framework.windows import pslist

import io
import logging
import os
import pefile

vollog = logging.getLogger(__name__)

IMAGE_DLL_CHARACTERISTICS_DYNAMIC_BASE = 0x0040
IMAGE_FILE_RELOCS_STRIPPED = 0x0001

We first handle the imports we'll need, plus the __pefile__ library for analyzing Portable Executable (__PE__) files. Now let's write a helper function to do that analysis:

In [ ]:
def check_aslr(pe): #[1]
    pe.parse_data_directories([
        pefile.DIRECTORY_ENTRY['IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG']
    ])
    dynamic = False
    stripped = False

    if (pe.OPTIONAL_HEADER.DllCharacteristics & #[2]
        IMAGE_DLL_CHARACTERISTICS_DYNAMIC_BASE):
        dynamic = True
    if pe.FILE_HEADER.Characteristics & IMAGE_FILE_RELOCS_STRIPPED: #[3]
        stripped = True
    if not dynamic or (dynamic and stripped): #[4]
        aslr = False
    else:
        aslr = True
    return aslr

We pass a PE file object to the __checkaslr__ function __[1]__, parse it, and then check for whether it has been compiled with the DYNAMIC base setting __[2]__ and whether the file relocation data has been stripped out __[3]__. If it's not dynamic, or was perhaps compiled as dynamic but stripped of its relocation data, then the PE file is not protected by ASLR __[4]__.

With the __check_aslr__ helper function ready to go, let's create our __AslrCheck__ class:

In [ ]:
class AslrCheck(interfaces.plugins.PluginInterface): #[1]

    @classmethod
    def get_requirements(cls):
        return [
            requirements.TranslationLayerRequirement( #[2]
                name='primary', description='Memory layer for the kernel',
                architectures=['Intel32', 'Intel64'],
            ),
            requirements.SymbolTableREequirement( #[3]
                name='nt_symbols', description='Windows kernel symbols'
            )
            requirements.PluginRequirement( #[4]
                name='pslist', plugin=pslist.PsList, version=(1, 0, 0)
            )
            requirements.ListRequirement( #[5]
                name='pid', element_type=int,
                description='Process ID to include (all others are excluded)',
                optional=True
            )
        ]

Step one of creating the plug-in is to inherit from the __PluginInterface__ object __[1]__. Next, define the requirements. You can get a good idea of what you need by reviewing other plug-ins. Every plug-in needs the memory layer, and we define that requirement first __[2]__. Along with the memory layer, we also need the symbols tables __[3]__ .You'll find these two requirements used by almost all plug-ins.

We'll also need the __pslist__ plug-in as a requirement in order to get all the processes from memory and re-create the PE file from the process __[4]__. Then we'll pass the re-created PE file from each process and examine it for ASLR protection.

We may want to check a single process given a process ID, so we create another optional setting that lets us pass in a list of process IDs to limit checking to just those processes __[5]__.

In [ ]:
    @classmethod
    def create_pid_filter(cls, pid_list: List[int] = None) -> Callable[[interfaces.objects.ObjectInterface], bool]:
        filter_func = lambda _: False
        pid_list = pid_list or []
        filter_list = [x for x in pid_list if x is not None]
        if filter_list:
            filter_func = lambda x: x.UniqueProcessId not in filter_list
        return filter_func

To handle the optional process ID, we use a class method to create a filter function that returns __False__ for every process ID in the list; that is, the question we're asking the filter function is whether to filter out a process, so we return __True__ only if the PID is not in the list:

In [ ]:
    def _generator(self, procs):
        pe_table_name = intermed.IntermediateSymbolTable.create( #[1]
            self.context,
            self.config_path,
            'windows',
            'pe',
            class_types=extensions.pe.class_types
        )

        procnames = list()
        for proc in procs:
            procname = proc.ImageFileName.cast('string',
                max_length=proc.ImageFileName.vol.count, errors='replace')
            if procname in procnames:
                continue
            procname.append(procname)

            proc_id = 'Unknown'
            try:
                proc_id = proc.UniqueProcessId
                proc_layer_name = proc.add_process_layer()
            except exceptions.InvalidAddressException as e:
                vollog.error(f'Process {proc_id}: invalid address {e} in layer '
                    '[e.layer_name]')
                continue

            peb = self.context.object( #[2]
                self.config['nt_symbols'] + constants.BANG + '_PEB',
                layer_name=proc_layer_name,
                offset=proc.Peb
            )

            try:
                dos_header = self.context.object(
                    pe_table_name + constants.BANG + '_IMAGE_DOS_HEADER',
                    offset=peb.ImageBaseAddress,
                    layer_name=proc_layer_name
                )
            except Exception as e:
                continue

            pe_data = io.BytesIO()
            for offset, data in dos_header.reconstruct():
                pe_data.seek(offset)
                pe_data.write(data)
            pe_data_raw = pe_data.getvalue() #[3]
            pe_data.close()

            try:
                pe = pefile:PE(data=pe_data_raw) #[4]
            except Exception as e:
                continue

            aslr = check_aslr(pe) #[5]

            yield (o, (proc_id, #[6]
                       procname,
                       format_hints.Hex(pe.OPTIONAL_HEADER.ImageBase),
                       aslr,
                       ))

We create a special data structure called __pe_table_name__ __[1]__ to use as we loop over each process in memory. Then we get the Process Environment Block (__PEB__) memory region associated with each process and put it into an object __[2]__. The __PEB__ is a data structure for the currnt process that contains a wealth of information on the process. We write that region into a file-like object (__pe_data__) __[3]__, create a PE object using the __pefile__ library __[4]__, and pass it to our __check_aslr__ helper method __[5]__. Finally, we yield the tuple of information containing the process ID, process name, memory address of the process, and a Boolean result from the ASLR protection check __[6]__. Now we create the __run__ method, which needs no arguments since all settings are populated in the config object:

In [ ]:
    def run(self):
        procs = pslist.PsList.list_processes(self.context, self.config['primary']), #[1]
            self.config['nt_symbols'], filter_func =
            self.create_pid_filter(self.config.get('pid', None))

        return renderers.TreeGrid([ #[2]
            ('PID', int),
            ('Filename', str),
            ('Base', format_hints.Hex),
            ('ASLR', bool),],
            self.generator(procs))

We get the list of processes using the __pslist__ plug-in __[1]__ and return the data from the generator using the __TreeGrid__ renderer __[2]__. The __TreeGrid__ renderer is used by many plug-ins. It ensures that we get one line of results for each process analyzed.

# *__Kicking the Tires__*

Let's take a look at one of the images made available at the Volatility site: Malware - Cridex. For your custom plug-in, provide the __-p__ switch with the path to your __plugins__ folder:

```
PS> vol -p .\plugins\windows -f cridex.vmem aslrcheck.AslrCheck
Volatility 3 Framework 1.2.0-beta.1
Progress: 0.00 Scanning primary2 using PdbSignatureScanner
PID Filename Base ASLR
368 smss.exe 0x48580000 False
584 csrss.exe 0x4a680000 False
608 winlogon.exe 0x1000000 False
652 services.exe 0x1000000 False
664 lsass.exe 0x1000000 False
824 svchost.exe 0x1000000 False
1484 explorer.exe 0x1000000 False
1512 spoolsv.exe 0x1000000 False
1640 reader_sl.exe 0x400000 False
788 alg.exe 0x1000000 False
1136 wuauclt.exe 0x400000 False
```

As you can see, this is a Windows XP machine, and there are no ASLR protections on any process.

Next is the result for a clean, up-to-date Windows 10 machine:

```
PS> vol -p .\plugins\windows -f WinDev2007Eval-Snapshot4.vmem aslrcheck.AslrCheck
Volatility 3 Framework 1.2.0-beta.1
Progress: 33.01 Scanning primary2 using PdbSignatureScanner
PID Filename Base ASLR
Offensive Forensics 183
316 smss.exe 0x7ff668020000 True
428 csrss.exe 0x7ff796c00000 True
500 wininit.exe 0x7ff7d9bc0000 True
568 winlogon.exe 0x7ff6d7e50000 True
592 services.exe 0x7ff76d450000 True
600 lsass.exe 0x7ff6f8320000 True
696 fontdrvhost.ex 0x7ff65ce30000 True
728 svchost.exe 0x7ff78eed0000 True

Volatility was unable to read a requested page:
Page error 0x7ff65f4d0000 in layer primary2_Process928 (Page Fault at entry 0xd40c9d88c8a00400
in page entry)

* Memory smear during acquisition (try re-acquiring if possible)
* An intentionally invalid page lookup (operating system protection)
* A bug in the plugin/volatility (re-run with -vvv and file a bug)

No further results will be produced
```

Not too much to see here. Every listed process is protected by ASLR. However, we also see a memory smear. A __memory smear__ occurs when the contents of the memory changes as the memory image is taken. That results in the memory table descriptions not matching the memory itself; alternatively, the virtual memory pointers may reference invalid data. Hacking is hard. As the error description says, you can try reacquiring the image (finding or creating a new snapshot).

Let's check the PassMark Windows 10 sample memory image:

```
PS> vol -p .\plugins\windows -f WinDump.mem aslrcheck.AslrCheck
Volatility 3 Framework 1.2.0-beta.1
Progress: 0.00 Scanning primary2 using PdbSignatureScanner
PID Filename Base ASLR
356 smss.exe 0x7ff6abfc0000 True
2688 MsMpEng.exe 0x7ff799490000 True
2800 SecurityHealth 0x7ff6ef1e0000 True
5932 GoogleCrashHan 0xed0000 True
5380 SearchIndexer. 0x7ff6756e0000 True
3376 winlogon.exe 0x7ff65ec50000 True
6976 dwm.exe 0x7ff6ddc80000 True
9336 atieclxx.exe 0x7ff7bbc30000 True
9932 remsh.exe 0x7ff736d40000 True
2192 SynTPEnh.exe 0x140000000 False
7688 explorer.exe 0x7ff7e7050000 True
7736 SynTPHelper.ex 0x7ff7782e0000 True
```

Nearly all processes are protected. Only the single process __SynTPEnh.exe__ isn't ASLR protected. An online search shows that this is a software component of Synaptics Pointing Device, probably for touch screens. As long as that process is installed in _C:\Program Files_, it's probably okay, but it may be worth fuzzing later on.

In this chapter, you saw that you can leverage the power of the Volatility framework to find more information about a user's behavior and connections as well as to analyze data on any process running memory. You can use that information to better understand the target user and machine as well as to understand the mindset of a defender.

# *__Onward!__*

You should have noticed by now that Python is a great language for hacking, especially when you consider the many libraries and Python-based frameworks you have available. While hackers have a plethora of tools, there's really no substitute for coding your own tools, because this gives you a deeper understanding of what those other tools are doing.

Go ahead and quickly code up a custom tool for your special requirements. Whether it's an SSH client for Windws, a web scaper, or a command-and-control system, Python has you covered.